In [ ]:
import pandas as pd
import numpy as np
from google.colab import drive
import re
import random
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
import nltk

#for model-building
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, f1_score, accuracy_score, confusion_matrix
from sklearn.metrics import roc_curve, auc, roc_auc_score

# bag of words
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
#for word embedding
import gensim
from gensim.models import Word2Vec

In [ ]:
drive.mount('/content/drive/')

%cd drive/MyDrive/Text Mining Project

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
[Errno 2] No such file or directory: 'drive/MyDrive/Text Mining Project'
/content/drive/MyDrive/Text Mining Project


In [ ]:
file_path = 'datasetV2.csv'

df_train = pd.read_csv(file_path)

In [ ]:
df_train.head()

,text,label
0,The in-app chat support feature is incredibly ...,1
1,The ability to track multiple orders simultane...,1
2,I suggest adding a feature to customize delive...,1
3,Providing estimated delivery times for each re...,1
4,Offering discounts for frequent users would en...,1


## Preprocessing

### Clean Text

In [ ]:
def clean_text(text):
    # Remove special characters and extra whitespace
    cleaned_text = re.sub(r'[^A-Za-z0-9\s\.]', '', text)
    # Convert multiple whitespace characters into a single space
    cleaned_text = re.sub(r'\s+', ' ', cleaned_text)
    # Convert text to lowercase
    cleaned_text = cleaned_text.lower()
    return cleaned_text

In [ ]:
df_train['text'] = df_train['text'].apply(clean_text)

In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:

df_train['clean_text_tok']=[nltk.word_tokenize(i) for i in df_train['text']] #convert preprocessed sentence to tokenized sentence
model = Word2Vec(df_train['clean_text_tok'],min_count=1)  #min_count=1 means word should be present at least across all documents,


w2v = dict(zip(model.wv.index_to_key, model.wv.vectors))

#for converting sentence to vectors/numbers from word vectors result by Word2Vec
class MeanEmbeddingVectorizer(object):
    def __init__(self, word2vec):
        self.word2vec = word2vec
        # if a text is empty we should return a vector of zeros
        # with the same dimensionality as all the other vectors
        self.dim = len(next(iter(word2vec.values())))

    def fit(self, X, y):
        return self

    def transform(self, X):
        return np.array([
            np.mean([self.word2vec[w] for w in words if w in self.word2vec]
                    or [np.zeros(self.dim)], axis=0)
            for words in X
        ])


## Naive Bayes

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(df_train["text"],
                                                  df_train["label"],
                                                  test_size=0.2,
                                                  shuffle=True)

In [ ]:
X_train_tok= [nltk.word_tokenize(i) for i in X_train]  #for word2vec
X_val_tok= [nltk.word_tokenize(i) for i in X_val]      #for word2vec

In [ ]:
#TF-IDF
# Convert x_train to vector since model can only run on numbers and not words- Fit and transform
tfidf_vectorizer = TfidfVectorizer(use_idf=True)
X_train_vectors_tfidf = tfidf_vectorizer.fit_transform(X_train) #tfidf runs on non-tokenized sentences unlike word2vec
# Only transform x_test (not fit and transform)
X_val_vectors_tfidf = tfidf_vectorizer.transform(X_val) #Don't fit() your TfidfVectorizer to your test data: it will

#Word2vec
# Fit and transform
modelw = MeanEmbeddingVectorizer(w2v)
X_train_vectors_w2v = modelw.transform(X_train_tok)
X_val_vectors_w2v = modelw.transform(X_val_tok)

In [ ]:
#FITTING THE CLASSIFICATION MODEL using Naive Bayes(tf-idf)
#It's a probabilistic classifier that makes use of Bayes' Theorem, a rule that uses probability to make predictions based on prior knowledge of conditions that might be related. This algorithm is the most suitable for such large dataset as it considers each feature independently, calculates the probability of each category, and then predicts the category with the highest probability.

nb_tfidf = MultinomialNB()
nb_tfidf.fit(X_train_vectors_tfidf, y_train)  #model

#Predict y value for test dataset
y_predict = nb_tfidf.predict(X_val_vectors_tfidf)
y_prob = nb_tfidf.predict_proba(X_val_vectors_tfidf)[:,1]


print(classification_report(y_val,y_predict))
print('Confusion Matrix:',confusion_matrix(y_val, y_predict))

auc_score = roc_auc_score(y_val, y_predict)
print(f'AUC Score: {auc_score}')

              precision    recall  f1-score   support

           0       1.00      0.02      0.05       125
           1       0.60      1.00      0.75       185

    accuracy                           0.61       310
   macro avg       0.80      0.51      0.40       310
weighted avg       0.76      0.61      0.47       310

Confusion Matrix: [[  3 122]
 [  0 185]]
AUC Score: 0.512
